In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
from PIL import Image
import os
from sklearn.model_selection import train_test_split

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = self._get_image_paths()

    def _get_image_paths(self):
        image_paths = []
        for filename in os.listdir(self.root_dir):
            if filename.endswith(".jpg"):
                image_paths.append(os.path.join(self.root_dir, filename))
        return image_paths

    def _is_tampered(self, filename):
        return 'tamp' in filename

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = 1 if self._is_tampered(img_path) else 0  # 1 for tampered, 0 for real
        return img, label


In [4]:
root_dir = '/content/drive/MyDrive/MICC-F220/MICC-F220'
batch_size = 32

In [5]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the required input size of your model
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

In [6]:
# Create dataset
dataset = CustomDataset(root_dir, transform=transform)

In [7]:
# Split dataset into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)  # Split training data further for validation

In [8]:
print("Size of train_data:", len(train_data))
print("Size of validation_data:", len(val_data))
print("Size of test_data:", len(test_data))

Size of train_data: 176
Size of validation_data: 22
Size of test_data: 22


In [9]:
# Create dataloaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [10]:
vit = torchvision.models.vit_b_16(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 142MB/s]


In [11]:
total_params = sum(p.numel() for p in vit.parameters())
total_params

86567656

In [12]:
import torch.optim as optim
optimizer = optim.Adam(vit.parameters(), lr=0.001)

In [13]:
# Define your loss function
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [14]:
# Define the device to run the model on (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
vit.to(device)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [18]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=8):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        # Training phase
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Calculate training loss and accuracy
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct / total

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_labels)
                val_running_loss += val_loss.item() * val_inputs.size(0)
                _, val_predicted = val_outputs.max(1)
                val_total += val_labels.size(0)
                val_correct += val_predicted.eq(val_labels).sum().item()

        # Calculate validation loss and accuracy
        epoch_val_loss = val_running_loss / len(val_loader.dataset)
        epoch_val_acc = val_correct / val_total

        # Print epoch statistics
        print(f"Epoch {epoch + 1}/{num_epochs}, "
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}, "
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    print('Training complete')


num_epochs = 10


train_model(vit, train_loader, val_loader, criterion, optimizer, device, num_epochs)


Epoch 1/10, Train Loss: 0.4992, Train Acc: 0.7898, Val Loss: 0.8841, Val Acc: 0.5000
Epoch 2/10, Train Loss: 0.5147, Train Acc: 0.7443, Val Loss: 0.4253, Val Acc: 0.8636
Epoch 3/10, Train Loss: 0.4082, Train Acc: 0.8295, Val Loss: 0.5000, Val Acc: 0.7273
Epoch 4/10, Train Loss: 0.3476, Train Acc: 0.8807, Val Loss: 0.5060, Val Acc: 0.7727
Epoch 5/10, Train Loss: 0.3567, Train Acc: 0.8807, Val Loss: 0.2370, Val Acc: 0.9545
Epoch 6/10, Train Loss: 0.3234, Train Acc: 0.8523, Val Loss: 0.3233, Val Acc: 0.8636
Epoch 7/10, Train Loss: 0.3445, Train Acc: 0.8750, Val Loss: 0.3391, Val Acc: 0.9091
Epoch 8/10, Train Loss: 0.2883, Train Acc: 0.8807, Val Loss: 0.2943, Val Acc: 0.8636
Epoch 9/10, Train Loss: 0.2722, Train Acc: 0.8920, Val Loss: 0.3328, Val Acc: 0.8182
Epoch 10/10, Train Loss: 0.2243, Train Acc: 0.9318, Val Loss: 0.2691, Val Acc: 0.9545
Training complete


In [19]:
def test_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # No need to calculate gradients during testing
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss = running_loss / len(test_loader.dataset)
    test_acc = correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [20]:
test_model(vit, test_loader, criterion, device)

Test Loss: 0.6530, Test Accuracy: 0.7727


In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion_mat = confusion_matrix(true_labels, predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(confusion_mat)

#Assuming you have already defined your test_loader and model
evaluate_model(vit, test_loader, device)



Accuracy: 0.7727
Precision: 0.6429
Recall: 1.0000
F1 Score: 0.7826
Confusion Matrix:
[[8 5]
 [0 9]]
